###### 2020-11-22 일요일

# 01_텍스트데이터토큰화 & 토큰사전저장

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import re
import json

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## 1. 데이터 불러오기

In [ ]:
d1 = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data1(2020-11-17).csv')
d2 = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data2(2020-11-17).csv')
# division_data3 경우는 인코딩안해주면 에러뜸 ㅠ
d3 = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data3(2020-11-17).csv', encoding='cp949')
# division_data4 경우는 read_excel로 불러와야 에러안뜸 ㅠ
d5 = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data5(2020-11-17).csv')
d6 = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data6(재원).csv')

In [ ]:
df_list = [d1, d2, d3, d5, d6]

all_df = d1
for df in df_list[1:]:
    all_df = pd.concat([all_df, df])

In [ ]:
all_df.drop(['Unnamed: 0', '출처'], axis=1, inplace=True)

index = all_df['악플여부'].isna()
raw_df = all_df[~index]

raw_df = raw_df.reset_index()
raw_df.drop(['index'], inplace=True, axis=1)

In [ ]:
copy_df = raw_df.copy()

In [ ]:
# 고정 시드값 지정
seed = 123

# 댓글 길이 지정
comment_len = 400

##### 악플여부 라벨이 1인경우 제거
 - 악플여부 라벨이 1인경우는 사람에 따라 불쾌할수도 있고 아닐수도 있기 때문에 기준을 매기기가 애매하다. 그러므로 객관적인 기준을 세울 수 없기 때문에 제거한다

In [ ]:
index = copy_df['악플여부'] == 1
copy_df = copy_df[~index]

In [ ]:
index = copy_df['악플여부'] == 2
copy_df.loc[index, '악플여부'] = 1

In [ ]:
copy_df['악플여부'] = copy_df['악플여부'].apply(int)

In [ ]:
copy_df['악플여부'].value_counts()

In [ ]:
copy_df['댓글'] = copy_df['댓글'].apply(str)

## 2. 토큰화 & 저장
 - Train set, Test set을 나눈다
 - Train set을 기준으로 음절 토큰 사전을 만든다
 - Train set / Test set / 음절토큰사전을 저장한다
    - 이렇게 저장하는 이유는? : 매번 음절토큰사전을 만드는데 시간이 많이 소요되기 때문에 한번 만들어놓고 재사용하여 시간을 절약!


In [ ]:
def tokenization_save(df):

    feature = df['댓글']
    label = df['악플여부']

    X_train, X_test, y_train, y_test = train_test_split(feature, label, 
                                                    test_size=0.2,
                                                    random_state=seed)
    X_train_split = X_train.apply(list).tolist()
    X_train_token_list = sum(X_train_split, [])
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train_token_list)
    
    word_index_vocab = tokenizer.word_index

    return X_train, X_test, y_train, y_test, word_index_vocab


In [ ]:
def train_test_save(X_train, X_test, y_train, y_test) :
    save_df_list = [X_train, X_test, y_train, y_test]
    file_names = ['X_train', 'X_test', 'y_train', 'y_test']
    for idx, set in enumerate(save_df_list):
        set.to_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/' + file_names[idx] + '.csv', encoding='UTF-8_SIG')


def save_word_index(word_index_vocab) :
    json_file = json.dumps(word_index_vocab, ensure_ascii=False)
    f = open('/content/drive/MyDrive/[final_project]_악플원정대/data/word_index_vocab.json', 'w')
    f.write(json_file)
    f.close()

In [ ]:
X_train, X_test, y_train, y_test, word_index_vocab = tokenization_save(copy_df)

In [ ]:
# ' '(공백)도 사전에 포함 
word_index_vocab[' '] = len(word_index_vocab) + 1

In [ ]:
train_test_save(X_train, X_test, y_train, y_test)
save_word_index(word_index_vocab)